# 🎯 Amazon ML Challenge 2025 - Product Price Prediction (Google Colab)

**Goal:** Predict product prices using catalog content (text descriptions)

**Evaluation Metric:** SMAPE (Symmetric Mean Absolute Percentage Error)

**Strategy:**
1. Advanced Feature Engineering from text
2. Multiple Gradient Boosting Models
3. Ensemble for maximum performance
4. Hyperparameter optimization

---

## 📌 **Colab Setup Instructions:**

### **Option A: Upload Files Directly**
Run the "File Upload" cell below and upload `train.csv` and `test.csv`

### **Option B: Use Google Drive**
1. Upload your dataset folder to Google Drive
2. Run the "Mount Google Drive" cell
3. Update the `DATASET_FOLDER` path

---

## 🔧 Colab Setup - Choose ONE option below

### Option A: Upload Files Directly (Run this cell)

In [ ]:
# OPTION A: Direct file upload
from google.colab import files
import os

# Create dataset folder
os.makedirs('dataset', exist_ok=True)

print("📤 Please upload train.csv and test.csv")
print("   (You can select multiple files at once)\n")

uploaded = files.upload()

# Move files to dataset folder
for filename in uploaded.keys():
    os.rename(filename, f'dataset/{filename}')
    print(f"✅ Uploaded: {filename}")

# Set dataset path
DATASET_FOLDER = 'dataset/'

# Verify files
import pandas as pd
train = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))

print(f"\n✅ Setup complete!")
print(f"   Training samples: {len(train):,}")
print(f"   Test samples: {len(test):,}")

### Option B: Mount Google Drive (Alternative)

In [ ]:
# OPTION B: Use Google Drive
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Update this path to where you uploaded your dataset in Google Drive
# Example: If you created a folder "ML_Challenge" in "My Drive" with dataset inside:
DATASET_FOLDER = '/content/drive/MyDrive/ML_Challenge/dataset/'

# Verify files
import pandas as pd
train = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))

print(f"✅ Google Drive mounted!")
print(f"   Training samples: {len(train):,}")
print(f"   Test samples: {len(test):,}")

## 📦 Part 1: Install & Import Libraries

In [ ]:
# Install required packages (Colab has most, but let's ensure)
!pip install -q lightgbm xgboost scikit-learn

print("✅ Packages installed!")

In [ ]:
# Core libraries
import os
import re
import warnings
import numpy as np
import pandas as pd
from pathlib import Path

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# NLP & Text Processing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

# Preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score

# Models
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
plt.style.use('seaborn-v0_8-darkgrid')

# Random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ All libraries imported successfully!")

## 📊 Part 2: Load & Explore Data

In [ ]:
# Load datasets (DATASET_FOLDER already set in setup cell)
train = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))

print(f"📈 Training data shape: {train.shape}")
print(f"📋 Test data shape: {test.shape}")
print(f"\n{'='*60}\n")

print("🔍 First few rows:")
display(train.head(3))

print(f"\n{'='*60}\n")
print("📊 Data Info:")
print(train.info())

In [ ]:
# Price statistics and distribution
print("💰 Price Statistics:")
print(train['price'].describe())

print(f"\n📊 Price Range: ${train['price'].min():.2f} - ${train['price'].max():.2f}")
print(f"📊 Price Median: ${train['price'].median():.2f}")
print(f"📊 Price Mean: ${train['price'].mean():.2f}")

# Visualize price distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Original price distribution
axes[0].hist(train['price'], bins=100, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Price')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Price Distribution (Original)')
axes[0].axvline(train['price'].median(), color='red', linestyle='--', label=f'Median: ${train["price"].median():.2f}')
axes[0].legend()

# Log-transformed price
axes[1].hist(np.log1p(train['price']), bins=100, edgecolor='black', alpha=0.7, color='green')
axes[1].set_xlabel('Log(Price + 1)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Price Distribution (Log-transformed)')

# Box plot
axes[2].boxplot(train['price'], vert=True)
axes[2].set_ylabel('Price')
axes[2].set_title('Price Box Plot')

plt.tight_layout()
plt.show()

print(f"\n⚠️ Number of outliers (price > $500): {(train['price'] > 500).sum()}")

In [ ]:
# Examine catalog content
print("📝 Sample Catalog Content:\n")
print("="*80)
for i in range(3):
    print(f"\n🔹 Sample {i+1} (Price: ${train['price'].iloc[i]:.2f})")
    print("-"*80)
    print(train['catalog_content'].iloc[i])
    print("="*80)

## 🛠️ Part 3: Advanced Feature Engineering

This is the **MOST CRITICAL** part for high performance!

In [ ]:
def extract_advanced_features(df):
    """
    Extract comprehensive features from catalog_content
    This is where we WIN the competition!
    """
    df = df.copy()
    
    # 1. Basic Text Features
    df['text_length'] = df['catalog_content'].str.len()
    df['word_count'] = df['catalog_content'].str.split().str.len()
    df['avg_word_length'] = df['catalog_content'].apply(lambda x: np.mean([len(word) for word in str(x).split()]))
    df['line_count'] = df['catalog_content'].str.count('\n') + 1
    
    # 2. Extract Value and Unit (KEY FEATURES)
    # Pattern: "Value: 72.0" or "Value: 32.0"
    df['value'] = df['catalog_content'].str.extract(r'Value:\s*([\d.]+)', expand=False).astype(float)
    df['value'] = df['value'].fillna(1.0)  # Default to 1 if missing
    
    # Extract Unit (Oz, Ounce, Pound, Count, etc.)
    df['unit'] = df['catalog_content'].str.extract(r'Unit:\s*(\w+)', expand=False)
    df['unit'] = df['unit'].fillna('Unknown')
    
    # 3. Extract Pack Size (e.g., "Pack of 6", "Pack of 12")
    df['pack_size'] = df['catalog_content'].str.extract(r'[Pp]ack of (\d+)', expand=False).astype(float)
    df['pack_size'] = df['pack_size'].fillna(1.0)
    
    # 4. Total Quantity (Value * Pack Size)
    df['total_quantity'] = df['value'] * df['pack_size']
    
    # 5. Extract Brand Name (first meaningful word from Item Name)
    def extract_brand(text):
        try:
            item_name = re.search(r'Item Name:\s*(.+?)(?:\n|$)', str(text))
            if item_name:
                name = item_name.group(1).strip()
                # Get first 1-2 words as brand
                words = name.split()[:2]
                return ' '.join(words) if words else 'Unknown'
        except:
            pass
        return 'Unknown'
    
    df['brand'] = df['catalog_content'].apply(extract_brand)
    
    # 6. Number of Bullet Points (more features = premium product?)
    df['num_bullet_points'] = df['catalog_content'].str.count('Bullet Point')
    
    # 7. Keyword Features (Premium indicators)
    premium_keywords = ['organic', 'premium', 'deluxe', 'professional', 'pro', 'ultra', 'plus']
    df['is_premium'] = df['catalog_content'].str.lower().str.contains('|'.join(premium_keywords)).astype(int)
    
    # 8. Product Category Hints
    food_keywords = ['sauce', 'cookie', 'snack', 'food', 'flavor', 'taste', 'ingredient']
    electronics_keywords = ['battery', 'electronic', 'digital', 'wireless', 'cable', 'charger']
    beauty_keywords = ['beauty', 'skin', 'hair', 'cosmetic', 'lotion', 'cream']
    
    df['is_food'] = df['catalog_content'].str.lower().str.contains('|'.join(food_keywords)).astype(int)
    df['is_electronics'] = df['catalog_content'].str.lower().str.contains('|'.join(electronics_keywords)).astype(int)
    df['is_beauty'] = df['catalog_content'].str.lower().str.contains('|'.join(beauty_keywords)).astype(int)
    
    # 9. Unit type encoding
    weight_units = ['Ounce', 'Oz', 'Pound', 'Lb', 'Gram', 'Kg']
    volume_units = ['Fl Oz', 'Liter', 'Gallon', 'Ml']
    
    df['is_weight_unit'] = df['unit'].isin(weight_units).astype(int)
    df['is_volume_unit'] = df['unit'].isin(volume_units).astype(int)
    df['is_count_unit'] = (df['unit'] == 'Count').astype(int)
    
    # 10. Numeric extraction from text
    df['numeric_count'] = df['catalog_content'].str.findall(r'\d+').str.len()
    
    print("✅ Advanced features extracted!")
    print(f"   - Total features created: {len([col for col in df.columns if col not in ['sample_id', 'catalog_content', 'image_link', 'price']])}")
    
    return df

# Apply feature engineering
print("🔧 Extracting features from training data...")
train_features = extract_advanced_features(train)

print("\n🔧 Extracting features from test data...")
test_features = extract_advanced_features(test)

print("\n📊 New feature columns:")
new_cols = [col for col in train_features.columns if col not in train.columns]
print(new_cols)

In [ ]:
# Visualize key features
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Value vs Price
axes[0, 0].scatter(train_features['value'], train_features['price'], alpha=0.3, s=10)
axes[0, 0].set_xlabel('Value (Quantity)')
axes[0, 0].set_ylabel('Price')
axes[0, 0].set_title('Value vs Price')

# Pack size vs Price
axes[0, 1].scatter(train_features['pack_size'], train_features['price'], alpha=0.3, s=10)
axes[0, 1].set_xlabel('Pack Size')
axes[0, 1].set_ylabel('Price')
axes[0, 1].set_title('Pack Size vs Price')

# Total quantity vs Price
axes[0, 2].scatter(train_features['total_quantity'], train_features['price'], alpha=0.3, s=10)
axes[0, 2].set_xlabel('Total Quantity')
axes[0, 2].set_ylabel('Price')
axes[0, 2].set_title('Total Quantity vs Price')

# Bullet points vs Price
axes[1, 0].boxplot([train_features[train_features['num_bullet_points'] == i]['price'].values 
                     for i in range(6)], labels=range(6))
axes[1, 0].set_xlabel('Number of Bullet Points')
axes[1, 0].set_ylabel('Price')
axes[1, 0].set_title('Bullet Points vs Price')

# Premium vs Non-premium
axes[1, 1].boxplot([train_features[train_features['is_premium'] == 0]['price'].values,
                     train_features[train_features['is_premium'] == 1]['price'].values],
                    labels=['Non-Premium', 'Premium'])
axes[1, 1].set_ylabel('Price')
axes[1, 1].set_title('Premium vs Non-Premium Products')

# Unit type distribution
unit_counts = train_features['unit'].value_counts().head(10)
axes[1, 2].barh(unit_counts.index, unit_counts.values)
axes[1, 2].set_xlabel('Count')
axes[1, 2].set_title('Top 10 Unit Types')

plt.tight_layout()
plt.show()

# Correlation with price
numeric_cols = train_features.select_dtypes(include=[np.number]).columns
correlations = train_features[numeric_cols].corr()['price'].sort_values(ascending=False)
print("\n🔗 Feature Correlations with Price:")
print(correlations.head(15))

## 📝 Part 4: Text Feature Extraction (TF-IDF)

Convert text to numerical features using TF-IDF

In [ ]:
# Clean text for TF-IDF
def clean_text(text):
    """Clean and preprocess text"""
    text = str(text).lower()
    # Remove special characters but keep important ones
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

train_features['cleaned_text'] = train_features['catalog_content'].apply(clean_text)
test_features['cleaned_text'] = test_features['catalog_content'].apply(clean_text)

print("✅ Text cleaned!")
print(f"\nOriginal text sample:\n{train_features['catalog_content'].iloc[0][:200]}...")
print(f"\nCleaned text sample:\n{train_features['cleaned_text'].iloc[0][:200]}...")

In [ ]:
# TF-IDF Vectorization
print("🔄 Creating TF-IDF features...")

# Configuration for optimal performance
tfidf = TfidfVectorizer(
    max_features=500,  # Top 500 most important words
    ngram_range=(1, 2),  # Unigrams and bigrams
    min_df=5,  # Word must appear in at least 5 documents
    max_df=0.8,  # Ignore words appearing in >80% of documents
    stop_words='english'
)

# Fit on combined data for consistency
all_text = pd.concat([train_features['cleaned_text'], test_features['cleaned_text']])
tfidf.fit(all_text)

# Transform
train_tfidf = tfidf.transform(train_features['cleaned_text'])
test_tfidf = tfidf.transform(test_features['cleaned_text'])

print(f"✅ TF-IDF matrix created: {train_tfidf.shape}")

# Dimensionality reduction with SVD (optional but helps)
print("\n🔄 Applying SVD for dimensionality reduction...")
svd = TruncatedSVD(n_components=100, random_state=RANDOM_STATE)
train_svd = svd.fit_transform(train_tfidf)
test_svd = svd.transform(test_tfidf)

print(f"✅ SVD features created: {train_svd.shape}")
print(f"   Explained variance ratio: {svd.explained_variance_ratio_.sum():.2%}")

# Convert to DataFrame
train_svd_df = pd.DataFrame(train_svd, columns=[f'svd_{i}' for i in range(100)])
test_svd_df = pd.DataFrame(test_svd, columns=[f'svd_{i}' for i in range(100)])

## 🎯 Part 5: Prepare Final Feature Set

In [ ]:
# Select features for modeling
numeric_features = [
    'text_length', 'word_count', 'avg_word_length', 'line_count',
    'value', 'pack_size', 'total_quantity', 'num_bullet_points',
    'is_premium', 'is_food', 'is_electronics', 'is_beauty',
    'is_weight_unit', 'is_volume_unit', 'is_count_unit', 'numeric_count'
]

categorical_features = ['unit', 'brand']

# Encode categorical features
print("🔄 Encoding categorical features...")
for col in categorical_features:
    le = LabelEncoder()
    # Combine train and test for consistent encoding
    combined = pd.concat([train_features[col], test_features[col]])
    le.fit(combined)
    
    train_features[f'{col}_encoded'] = le.transform(train_features[col])
    test_features[f'{col}_encoded'] = le.transform(test_features[col])

encoded_features = ['unit_encoded', 'brand_encoded']

# Combine all features
all_features = numeric_features + encoded_features

# Create final feature matrices
X_train_base = train_features[all_features].copy()
X_test_base = test_features[all_features].copy()

# Add SVD features
X_train = pd.concat([X_train_base.reset_index(drop=True), train_svd_df], axis=1)
X_test = pd.concat([X_test_base.reset_index(drop=True), test_svd_df], axis=1)

# Target variable (log-transform for better distribution)
y_train = np.log1p(train_features['price'].values)

print(f"\n✅ Final feature matrix prepared!")
print(f"   Training shape: {X_train.shape}")
print(f"   Test shape: {X_test.shape}")
print(f"   Target transformation: log(price + 1)")
print(f"\n📊 Feature list ({len(X_train.columns)} total):")
print(f"   - Numeric features: {len(numeric_features)}")
print(f"   - Encoded categorical: {len(encoded_features)}")
print(f"   - Text features (SVD): {len(train_svd_df.columns)}")

## 🎯 Part 6: Define SMAPE Metric

In [ ]:
def smape(y_true, y_pred):
    """
    Symmetric Mean Absolute Percentage Error
    Lower is better (0-200% range)
    """
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred)
    # Avoid division by zero
    denominator = np.where(denominator == 0, 1, denominator)
    return np.mean(diff / denominator) * 100

def evaluate_model(y_true, y_pred, model_name="Model"):
    """
    Comprehensive evaluation
    """
    # If predictions are log-transformed, convert back
    y_true_orig = np.expm1(y_true)
    y_pred_orig = np.expm1(y_pred)
    
    smape_score = smape(y_true_orig, y_pred_orig)
    rmse = np.sqrt(mean_squared_error(y_true_orig, y_pred_orig))
    mae = mean_absolute_error(y_true_orig, y_pred_orig)
    r2 = r2_score(y_true_orig, y_pred_orig)
    
    print(f"\n{'='*60}")
    print(f"📊 {model_name} Performance:")
    print(f"{'='*60}")
    print(f"🎯 SMAPE:        {smape_score:.4f}%  ⭐ (Primary Metric)")
    print(f"📉 RMSE:         ${rmse:.2f}")
    print(f"📉 MAE:          ${mae:.2f}")
    print(f"📈 R² Score:     {r2:.4f}")
    print(f"{'='*60}\n")
    
    return smape_score

print("✅ SMAPE metric defined!")

## 🤖 Part 7: Train Multiple Models

We'll train several models and ensemble them for maximum performance

In [ ]:
# Split data for validation
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=RANDOM_STATE
)

print(f"📊 Data split:")
print(f"   Training:   {X_tr.shape[0]} samples")
print(f"   Validation: {X_val.shape[0]} samples")
print(f"   Test:       {X_test.shape[0]} samples")

### Model 1: LightGBM (Primary Model) ⭐

In [ ]:
print("🚀 Training LightGBM model...\n")

# Optimized hyperparameters for performance
lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 63,
    'max_depth': 8,
    'min_child_samples': 20,
    'subsample': 0.8,
    'subsample_freq': 1,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'n_estimators': 2000,
    'random_state': RANDOM_STATE,
    'verbose': -1
}

# Train model
lgb_model = lgb.LGBMRegressor(**lgb_params)
lgb_model.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)]
)

# Predict
lgb_val_pred = lgb_model.predict(X_val)
lgb_test_pred = lgb_model.predict(X_test)

# Evaluate
lgb_smape = evaluate_model(y_val, lgb_val_pred, "LightGBM")

print(f"✅ LightGBM trained with {lgb_model.best_iteration_} iterations")

### Model 2: XGBoost ⭐

In [ ]:
print("🚀 Training XGBoost model...\n")

# Optimized hyperparameters
xgb_params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.05,
    'max_depth': 7,
    'min_child_weight': 3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 1.0,
    'n_estimators': 2000,
    'random_state': RANDOM_STATE,
    'tree_method': 'hist'
}

# Train model
xgb_model = xgb.XGBRegressor(**xgb_params)
xgb_model.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    verbose=False
)

# Predict
xgb_val_pred = xgb_model.predict(X_val)
xgb_test_pred = xgb_model.predict(X_test)

# Evaluate
xgb_smape = evaluate_model(y_val, xgb_val_pred, "XGBoost")

print(f"✅ XGBoost trained successfully!")

### Model 3: Ridge Regression (Baseline)

In [ ]:
print("🚀 Training Ridge Regression model...\n")

# Scale features for Ridge
scaler = RobustScaler()
X_tr_scaled = scaler.fit_transform(X_tr)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Train model
ridge_model = Ridge(alpha=10.0, random_state=RANDOM_STATE)
ridge_model.fit(X_tr_scaled, y_tr)

# Predict
ridge_val_pred = ridge_model.predict(X_val_scaled)
ridge_test_pred = ridge_model.predict(X_test_scaled)

# Evaluate
ridge_smape = evaluate_model(y_val, ridge_val_pred, "Ridge Regression")

print(f"✅ Ridge Regression trained successfully!")

## 🎭 Part 8: Model Ensemble

In [ ]:
# Weighted ensemble based on performance
print("\n🎭 Creating Ensemble Model...\n")

# Calculate weights inversely proportional to SMAPE
weights = 1 / np.array([lgb_smape, xgb_smape, ridge_smape])
weights = weights / weights.sum()  # Normalize

print(f"Ensemble weights:")
print(f"  LightGBM: {weights[0]:.3f}")
print(f"  XGBoost:  {weights[1]:.3f}")
print(f"  Ridge:    {weights[2]:.3f}")

# Ensemble predictions
ensemble_val_pred = (
    weights[0] * lgb_val_pred +
    weights[1] * xgb_val_pred +
    weights[2] * ridge_val_pred
)

ensemble_test_pred = (
    weights[0] * lgb_test_pred +
    weights[1] * xgb_test_pred +
    weights[2] * ridge_test_pred
)

# Evaluate ensemble
ensemble_smape = evaluate_model(y_val, ensemble_val_pred, "🎭 ENSEMBLE")

## 🎯 Part 9: Generate Final Predictions & Submission

In [ ]:
print("🎯 Generating final predictions for test set...\n")

# Convert log-predictions back to original scale
final_predictions = np.expm1(ensemble_test_pred)

# Ensure all predictions are positive
final_predictions = np.maximum(final_predictions, 0.01)

# Create submission DataFrame
submission = pd.DataFrame({
    'sample_id': test['sample_id'],
    'price': final_predictions
})

# Round prices to 2 decimal places
submission['price'] = submission['price'].round(2)

print("📊 Submission Statistics:")
print(submission['price'].describe())

print(f"\n✅ Sample predictions:\n")
print(submission.head(10))

# Verify submission format
assert len(submission) == len(test), "❌ Submission length mismatch!"
assert submission['sample_id'].equals(test['sample_id']), "❌ Sample IDs don't match!"
assert (submission['price'] > 0).all(), "❌ Found negative prices!"

print("\n✅ Submission format validated!")

In [ ]:
# Save submission file
output_path = 'test_out.csv'
submission.to_csv(output_path, index=False)

print(f"\n{'='*60}")
print(f"🎉 SUCCESS! Submission file saved to:")
print(f"   {output_path}")
print(f"{'='*60}")
print(f"\n📋 File Details:")
print(f"   - Total predictions: {len(submission):,}")
print(f"   - SMAPE on validation: {ensemble_smape:.4f}%")
print(f"   - Price range: ${submission['price'].min():.2f} - ${submission['price'].max():.2f}")
print(f"\n🚀 Ready to upload to the leaderboard!")

## 📥 Download Submission File

In [ ]:
# Download the submission file to your computer
from google.colab import files

files.download('test_out.csv')
print("✅ Downloading test_out.csv to your computer...")

---

## 🎉 Congratulations!

You've successfully built a complete ML pipeline in Google Colab!

**Next Steps:**
1. Download `test_out.csv` (cell above)
2. Upload to competition leaderboard
3. Check your SMAPE score
4. Iterate and improve!

**Ready for submission! 🚀**